# Obstacle avoidance block

## Imports

Import the script `thymio.py` and other libraries 

In [1]:
import os
import sys
import time
import serial

# Variable to know if Thymio has already been connected before or not (== 1 if it is the case)
try:
    CONNECC
except NameError:
    CONNECC = 0
    print('Thymio will be connected.')

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

# Print the path to Python3 executable
# print(sys.executable)

Thymio will be connected.


## Thymio connection

First connect Thymio to the computer on one USB port through the USB cable.

### Look-up table for ports:
Replace `/dev/cu.usbmodem142101` (or current port value) below by the correct computer port:

Lucas: `\\.\COM3` for USB-cable and `\\.\COM4` for USB-dongle

Emma: ` `

Elise: ` `

Océane: ` `

In [2]:
if CONNECC == 0:
    CONNECC = 1
    th = Thymio.serial(port="\\.\COM4", refreshing_rate=0.1)
    time.sleep(1)

## Tests

Testing communication with Thymio.

In [5]:
# Defines if tests should be run
TEST = 0

### Retrieve values

In [4]:
if TEST == 1:
    for i in range(10):
        print(th["prox.horizontal"])
        time.sleep(0.2)

[0, 0, 0, 0, 1279, 0, 0]
[0, 0, 0, 0, 1275, 0, 0]
[0, 0, 0, 0, 1268, 0, 0]
[0, 0, 0, 0, 1203, 0, 0]
[0, 0, 0, 0, 1283, 0, 0]
[0, 0, 0, 0, 1372, 0, 0]
[0, 0, 0, 0, 1376, 0, 0]
[0, 0, 0, 0, 1381, 0, 0]
[0, 0, 0, 0, 1378, 0, 0]
[0, 0, 0, 0, 1386, 0, 0]


### Set values

In [ ]:
if TEST == 1:
    th.set_var("motor.left.target", 100)
    th.set_var("motor.right.target", 100)

    time.sleep(1)

    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)

## Obstacle avoidance algorithm

The robot can be in two states: global navigation or obstacle avoidance when one is detected by the sensors (caution for not confounding walls from the map and unknown obstacles).

When in obstacle avoidance state, the robot accelerates the wheels close to the obstacles.

In [ ]:
th.variable_description()

In [139]:
# dir(th)
# th.decscription_variable()
# Conio
# th.set_var("mic.intensity", 0)

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
print(th["prox.ground.delta"])

[8, 8]


## Calibration Odometry

~ 0.0325 cm/s for 1 motor input unit of Thymio

rotation: ~9.32s for 360° with [100, -100] motor inputs => ~ 0.0067 rad/s for [1, -1] motor inputs

In [3]:
speed = []
omega = 2*3.1415926/9.32
omega_thymio = omega/100
print(omega_thymio)

0.006741615021459227


In [ ]:


delta_threshold = 300
square = 5 # cm

count = 0
current_square = 1
previous_square = current_square
change = []
vel = 100
start_time =  time.time()
#print(th["prox.ground.delta"])

while (count < 3):
    th.set_var("motor.left.target", vel)
    th.set_var("motor.right.target", vel)
    ground = th["prox.ground.delta"]
    if (time.time() - start_time) > 30:
        print('TIMED OUT')
        break
    if (ground[0] < delta_threshold and ground[1] < delta_threshold):
        current_square = 0
    else:
        current_square = 1
    if current_square != previous_square:
        start_time = time.time()
        change.append(time.time())
        previous_square = current_square
        count += 1

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)

first = change[0]
change = [x - first for x in change]
for i in range(0, len(change) - 1):
        speed.append(square/(change[i+1]-change[i])/vel) # [cm/s by Thymio velocity unit]

# print(change)
mean_speed = sum(speed)/len(speed)
print(mean_speed)

In [142]:
# Rotation odometry

start = time.time()
th.set_var("motor.left.target", 100)
th.set_var("motor.right.target", 2**16 - 100)
time.sleep(1)

while (stop - start) < 9.32:
    stop = time.time()

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
print(stop - start)

9.320163011550903


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\lucas\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\lucas\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\lucas\Documents\EPFL\MA1\MICRO-452 Basics of mobile robotics\Project\src\Thymio.py", line 340, in do_refresh
    self.get_variables()
  File "C:\Users\lucas\Documents\EPFL\MA1\MICRO-452 Basics of mobile robotics\Project\src\Thymio.py", line 499, in get_variables
    self.send(msg)
  File "C:\Users\lucas\Documents\EPFL\MA1\MICRO-452 Basics of mobile robotics\Project\src\Thymio.py", line 456, in send
    self.io.write(msg.serialize())
  File "C:\Users\lucas\Anaconda3\lib\site-packages\serial\serialwin32.py", line 315, in write
    raise SerialException("WriteFile failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: WriteFile failed (PermissionError(13, 'Le périphérique ne rec

In [83]:
print(th["prox.horizontal"])

[2923, 2946, 0, 0, 0, 0, 0]


In [ ]:
# Initialisation for testing the obstacle avoidance algorithm, Thymio is going straight towards its goal.
Speed0 = 100
ObstacleSpeedGain = 2
ObstacleThresholdLeft = 10
ObstacleThreshholdRight = 20

State = 0
Sensors = [0,0,0,0,0,0,0] # proximity sensors array
Now = time.time() # Actual time in seconds since 1st of Janurary 1970
Timer = Now

while:
    Timer = time.time()

dir